In [1]:
import torch
import spacy
import pandas as pd
import numpy as np
import re

In [2]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
torch.set_default_device(device)
print(f"Using device = {torch.get_default_device()}")

Using device = cpu


In [3]:
file_path = 'MINDlarge_train/news.tsv'
column_names = [
    "News ID",
    "Category",
    "SubCategory",
    "Title",
    "Abstract",
    "URL",
    "Title Entities",
    "Abstract Entities"

]
df = pd.read_csv(file_path, sep='\t', names=column_names, header=None)
df.head()


,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [4]:
df.shape

(101527, 8)

In [5]:
df['Title'].isnull().sum()

0

In [6]:
df['Abstract'].fillna('', inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_15376\3675929783.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Abstract'].fillna('', inplace=True)


In [7]:
def combine_title_abstract(row):
    title = re.sub(r'\s+', ' ', str(row['Title'])).strip()
    abstract = re.sub(r'\s+', ' ', str(row['Abstract'])).strip()
    return f"{title}\n{abstract}"

df['news'] = df.apply(combine_title_abstract, axis=1)

In [8]:
df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities,news
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],"The Brands Queen Elizabeth, Prince Charles, an..."
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...",Walmart Slashes Prices on Last-Generation iPad...
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",50 Worst Habits For Belly Fat\nThese seemingly...
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[],Dispose of unwanted prescription drugs during ...
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",The Cost of Trump's Aid Freeze in the Trenches...


In [9]:
print(df['news'][:200])

0      The Brands Queen Elizabeth, Prince Charles, an...
1      Walmart Slashes Prices on Last-Generation iPad...
2      50 Worst Habits For Belly Fat\nThese seemingly...
3      Dispose of unwanted prescription drugs during ...
4      The Cost of Trump's Aid Freeze in the Trenches...
                             ...                        
195    The Oldsmobile Toronado - was it the ultimate ...
196    Flying while pregnant? Here's what you need to...
197    25 Toys That the 1-Year-Old in Your Life Will ...
198    Patriots' Kyle Van Noy amused over Sam Darnold...
199    With impeachment inquiry 'concerns' getting ad...
Name: news, Length: 200, dtype: object


In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
def filter(doc):
    tokenized_doc = nlp(doc)
    return [t.lemma_ for t in tokenized_doc if
          not t.is_space and \
          not t.is_stop and \
          t.pos_ in ['NOUN', 'VERB', 'ADJ']]

In [13]:
df['tokens'] = df['news'].apply(filter)

In [14]:
df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities,news,tokens
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],"The Brands Queen Elizabeth, Prince Charles, an...","[shop, notebook, jacket, royal, live]"
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...",Walmart Slashes Prices on Last-Generation iPad...,"[slash, new, release, bring, big, deal, year, ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",50 Worst Habits For Belly Fat\nThese seemingly...,"[bad, habit, harmless, habit, hold, keep, shed..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[],Dispose of unwanted prescription drugs during ...,"[unwanted, prescription, drug, day]"
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",The Cost of Trump's Aid Freeze in the Trenches...,"[aid, freeze, peek, parapet, sand, bag, line, ..."


In [15]:
df.to_csv('news_topic.csv', index=False)